<a href="https://colab.research.google.com/github/nalewkoz/neural-bifurcations/blob/main/Ising2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the cell below to load helper functions.

In [ ]:
#@title
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import time
from matplotlib import rc
rc('font',**{'size':20})

## Not used at the moment:
precision = torch.double

## Check if a GPU available
if(torch.cuda.is_available()):
  print("GPU available: "+torch.cuda.get_device_name())
  dev = "cuda:0"
else:
  print("No GPU available... Using CPU.")
  dev = "cpu:0"

device = torch.device(dev)
torch.cuda.empty_cache()

def initState(params):
    #X = torch.empty((params["Nx"], params["Ny"]), dtype = torch.bool) ## THIS DOESN'T REALLY WORK AS EXPECTED! DOESN'T SEEM TO CONVERGE TO THERMAL EQ. AVOID TORCH.EMPTY!
#   X = torch.zeros((params["Nx"], params["Ny"]), dtype = torch.bool)
    X = torch.rand((params["Nx"], params["Ny"])) < (1 + params["M0"])/2
    return X

def chooseRandomSpin(params):
    #return (torch.randint(params["Nx"],(1,1)), torch.randint(params["Nx"],(1,1)))
    return (np.random.randint(0,params["Nx"]), np.random.randint(0,params["Ny"]) )

def get_neighbors(spinInds, params):
    nb = []
    ## THIS IS WRONG:
    ## leaving just to remember my stupid mistake.. It creates two independent lattices! 
    #for dx in (-1,1):
    #    for dy in (-1,1):
    #        sI = ( (spinInds[0] + dx)%params["Nx"], (spinInds[1] + dy)%params["Ny"] )
    #        nb.append(sI)
    for dx in (-1,1):
        sI = ( (spinInds[0] + dx)%params["Nx"], spinInds[1] )
        nb.append(sI)
        sI = ( spinInds[0], (spinInds[1] + dx)%params["Ny"] )
        nb.append(sI)
    return nb

def singleSpinStep(X, spinInds, params):
    #print("single step")

    ## Metropolis-Hasting
    # calculate the local field
    heff = params["h"]
    for nI in get_neighbors(spinInds, params):
        heff = heff - params["J"]*(-1)**(X[nI])     # positive neighbors pull us up (positive heff)
    # calculate the current local energy of the spin of interest
    e0 = heff*(-1)**(X[spinInds])
    # change in the energy from flipping would be: -e0-e0 = -2e0
    de = -2*e0
    # check if a flip needed:
    if de < 0: # new energy lower than old, flip!
        X[spinInds] = ~ X[spinInds]
    else:
        if np.random.rand() < np.exp( - de/params["T"] ):
            X[spinInds] = ~ X[spinInds]  
    # Note: I could write much shorter code (noticing that in the first case of de < 0 the second condition is always fullfilled), but it is going to be slower (likely)

def magnetization(X, params):
    nPositive = torch.sum(X, (0,1))
    return 2*nPositive/(params["Nx"]*params["Ny"]) - 1

def simulate(params, tsteps = 10000, Nhist = 100):
    print("Simulating... ",end='')
    # when take a screenshot:
    steps_ss = torch.linspace(0,tsteps,Nhist).to(int)
    ss_ind   = 0
    # variables to be returned
    Mhist = []
    
    X = initState(params)
    Xhist = torch.zeros((Nhist,params["Nx"],params["Ny"]), dtype=torch.bool)
    Thist = []
    Mhist.append( magnetization(X,params) )
      
    # loop over "time"
    for t in range(tsteps):
        # Add screenshot if it's time:
        if steps_ss[ss_ind]==t:
            Xhist[ss_ind,:,:] = X
            Thist.append(t)
            ss_ind = ss_ind + 1
        # Simulate
        singleSpinStep(X, chooseRandomSpin(params), params)
        # Keep the history o magnetization for each step
        Mhist.append( magnetization(X,params) )

    # Check if we didn't skip the last time step
    if ss_ind < len(steps_ss):
        if steps_ss[ss_ind] == tsteps:
            Xhist[ss_ind,:,:] = X
            Thist.append(tsteps)

    print("DONE")

    return Mhist, Xhist

def draw_Mhist_spins(Mhist, Xhist, params):
    print("Drawing... ",end='')
    # Magnetization as a function of time:
    fig = plt.figure()
    plt.plot(range(len(Mhist)), Mhist)
    plt.xlabel("Time (step)")
    plt.ylabel("Magnetization")
    # Last state:
    fig3 = plt.figure()
    plt.imshow(X[0,:,:], vmin=0, vmax=1)
    plt.title("Initial state")
    plt.axis('off')
    # Last state:
    fig2 = plt.figure()
    plt.imshow(X[-1,:,:], vmin=0, vmax=1)
    plt.title("  End state  ")
    plt.axis('off')
    ## Sequence of images in a grid:
    #grid_img = torchvision.utils.make_grid(X.reshape((X.shape[0],1,X.shape[1],X.shape[1])), nrow=5)
    #fig3 = plt.figure(figsize=(20,20))
    #plt.imshow(grid_img[0,:,:])
    #plt.axis('off')
    print("DONE")


###Exercise 1
Run simulations for different control parameters $T$ (temperature) and $h$ (external field). Note the difference in the behavior of the system (magnetization $M$ in particular) above and below the critical temperature.  

Are the results affected by the value of the initial magnetization? Why?

$$M = \frac{1}{N}\sum\limits_{i=1}^N s_i \in [-1,1]$$ 

In [ ]:
# For J=1 the critical temperature is:
Tc = 2/np.log(1+np.sqrt(2))
print(f"Critical temperature according to Onsager's solution: {Tc:.2f}")

## General parameters
params = {
        "Nx":       int(4e1),   # number of rows
        "Ny":       int(4e1),   # number of columns
        "J":        1,          # interaction strength (nearest neighbors)
        "T":        Tc/2,     # temperature (kb = 1)
        "h":        0.0,          # external field
        "M0":       0.5          # initial magnetization
}

# SIMULATE:
Mhist, X = simulate(params, tsteps = int(1e5))

# DRAW:
draw_Mhist_spins(Mhist, X, params)


Run the cell below to generate an animation.

In [ ]:
#@title
from matplotlib import animation, rc
from IPython.display import HTML#, Image # For GIF
rc('animation', html='html5')

fig = plt.figure()
Xdata = torch.zeros_like(X[0,:,:])
im = plt.imshow(Xdata, interpolation='none', vmin=0, vmax=1)
plt.axis('off')

def init():
    #im.set_data(Xdata)
    #ax.axis('off')
    return [im]

def update(frame):
    im.set_array(X[frame,:,:])
    return [im]

anim = animation.FuncAnimation(fig, update, frames=X.shape[0],
                    init_func=init, blit=True)

HTML(anim.to_html5_video())

###Exercise 2
Fill in the gaps in the code below. Plot the equilibrium magnetization as a function of temperature for a fixed value of $h$. 

Note that we have to run multiple Monte Carlo simulations, which is time consuming. Start from a low number of time steps (e.g., $tsteps=1000$) to check your code. If everything works as expected, increase the number of time steps to $10^5$ (it will take approximately $3$ minutes to finish). 

- Is the transition sharp? Why?
- What happens if the number of time steps is too low?

In [ ]:
Ttab = np.linspace(Tc/2, 3*Tc/2, 10)    # Values of the temperature at which we want to run simulations
mtab = []                               # Magnetizations (a list of values for each temperature)

for T in Ttab:
    params["T"] = T
    Mhist, X = simulate(params, tsteps = int(1e5))
    m = ...
    mtab.append(m)

fig = plt.figure()
plt.plot([Ttab[0], Ttab[-1]], [0, 0],'k--')
plt.plot(Tc, 0,'ro')
plt.plot(..., ..., 'o')
plt.xlabel("Temperature")
plt.ylabel("Magnetization")
print("DONE")